Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# 1. Unimodalne autokodery

Zanim przejdziemy do właściwych reprezentacji wielomodalnych sprawdzimy jak sprawdzają się reprezentacje unimodalne, tutaj: obrazowe oraz tekstowe.

Z racji, że cechy obrazów pozyskaliśmy za pomocą sieci konwolucyjnej (ResNet50), a cechy tekstowe za pomocą modelu językowego (all-MiniLM-L6-v2), teraz będziemy wykorzystywać proste autokodery oparte o sieci MLP.

In [ ]:
import pandas as pd
import torch
from sklearn.decomposition import PCA

from src.dataset import DataModule
from src.downstream import evaluate_classification
from src.nn.unimodal import UnimodalAE
from src.train import extract_embeddings, train_model
from src.visualization import make_interactive_scatter_plot, visualize_most_similar

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./data/logs --port 6006

Określamy domyślne wartości hiperparametrów

In [ ]:
default_hparams = {
    "batch_size": 64,
    "num_epochs": 30,
    "hidden_dims": [256, 256, 256],
    "emb_dim": 128,
    "lr": 1e-3,
    "weight_decay": 5e-4,
}

Ładujemy dane dla uczenia modeli

In [ ]:
datamodule = DataModule(batch_size=default_hparams["batch_size"])

## Cechy wizualne
Najpierw uczymy autokoder unimodalny, który będzie wykorzystywać tylko informacje o cechach pozyskanych z obrazka mema.

In [ ]:
train_model(
    model_cls=UnimodalAE,
    hparams={
        "name": "ImageAE", 
        "data_dim": 2048, 
        "modality_name": "img_emb", 
        **default_hparams,
    },
    datamodule=datamodule,
)

Wykorzystując wyuczony model (`name` definiuje, który model chcemy wykorzystać), zapisujemy wektory reprezentacji używając tylko części kodera:

In [ ]:
unimodal_img_emb = extract_embeddings(
    model_cls=UnimodalAE, 
    name="ImageAE",
    datamodule=datamodule,
)

Wizualizujemy przestrzeń ukrytą:

In [ ]:
make_interactive_scatter_plot(
    title="Unimodal image embeddings",
    z_2d=PCA(n_components=2).fit_transform(unimodal_img_emb),
    df=datamodule.df["all"],
)

Dla wybranej próbki danych znajdujemy najbliższe instancje w przestrzeni reprezentacji. Zachęcamy przejrzenia innych przykładów.

In [ ]:
_ = visualize_most_similar(
    title="Most similar by image embedding",
    anchor_index=339,
    z=unimodal_img_emb,
    df=datamodule.df["all"],
)

## Cechy tekstowe
Analogicznie wyuczymy model unimodalnego autokodera, który wykorzystuje tylko cechy pozyskane z napisów umieszczonych na memie.

In [ ]:
train_model(
    model_cls=UnimodalAE,
    hparams={
        "name": "TextAE",
        "data_dim": 384,
        "modality_name": "text_emb",
        **default_hparams,
    },
    datamodule=datamodule,
)

In [ ]:
unimodal_text_emb = extract_embeddings(
    model_cls=UnimodalAE,
    name="TextAE",
    datamodule=datamodule,
)

In [ ]:
make_interactive_scatter_plot(
    title="Unimodal text embeddings",
    z_2d=PCA(n_components=2).fit_transform(unimodal_text_emb),
    df=datamodule.df["all"],
)

In [ ]:
_ = visualize_most_similar(
    title="Most similar by text embedding",
    anchor_index=339,
    z=unimodal_text_emb,
    df=datamodule.df["all"],
)

## Ewaluacja wektorów reprezentacji
Dotychczas ocena jakości reprezentacji była wykonywana oceniając grupowanie się przykładów w 2-wymiarowym rzucie przestrzeni reprezentacji oraz przeglądając podobne instancje. Wykorzystywany zbiór danych posiada również 5 niezależnych etykiet (`humour`, `motivational`, `offensive`, `sarcasm`, `sentiment`). Sprawdźmy zatem jak reprezentacje sprawdzają się w tych 5 zadaniach klasyfikacji.

In [ ]:
evaluate_classification(
    model_names=[(UnimodalAE, "ImageAE"), (UnimodalAE, "TextAE")],
    datamodule=datamodule,
)